<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/mistral_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 调用Mistral代理的函数


这个笔记本向您展示了如何使用我们的Mistral代理，它由函数调用功能驱动。


## 初始设置


让我们从导入一些简单的基本模块开始。

我们主要需要的是：
1. OpenAI API（使用我们自己的 `llama_index` LLM 类）
2. 一个用于保存对话历史记录的地方
3. 我们的代理程序可以使用的工具的定义。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-llms-mistralai
%pip install llama-index-embeddings-mistralai

In [ ]:
!pip install llama-index

In [ ]:
import json
from typing import Sequence, List

from llama_index.llms.mistralai import MistralAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

让我们为我们的代理人定义一些非常简单的计算器工具。


In [ ]:
def multiply(a: int, b: int) -> int:    """将两个整数相乘，并返回结果整数"""    return a * bmultiply_tool = FunctionTool.from_defaults(fn=multiply)

In [ ]:
def add(a: int, b: int) -> int:    """对两个整数进行相加，并返回结果整数"""    return a + badd_tool = FunctionTool.from_defaults(fn=add)

确保您设置了MISTRAL_API_KEY。否则，请明确指定`api_key`参数。


In [ ]:
llm = MistralAI(model="mistral-large-latest")

## 初始化 Mistral 代理


在这里，我们使用计算器函数初始化一个简单的Mistral代理。


In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = agent_worker.as_agent()

这是一个用Python编写的简单聊天程序。


In [ ]:
response = agent.chat("What is (121 + 2) * 5?")
print(str(response))

Added user message to memory: What is (121 + 2) * 5?
=== Calling Function ===
Calling function: add with args: {"a": 121, "b": 2}
=== Calling Function ===
Calling function: multiply with args: {"a": 123, "b": 5}
assistant: The result of (121 + 2) * 5 is 615.


In [ ]:
# 检查数据源print(response.sources)

[ToolOutput(content='123', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 2}}, raw_output=123), ToolOutput(content='615', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 123, 'b': 5}}, raw_output=615)]


### 异步聊天

同时，让我们重新启用并行函数调用，这样我们就可以同时调用两个`multiply`操作。


In [ ]:
# 启用并行函数调用agent_worker = FunctionCallingAgentWorker.from_tools(    [multiply_tool, add_tool],    llm=llm,    verbose=True,    allow_parallel_tool_calls=True,)agent = agent_worker.as_agent()response = await agent.achat("What is (121 * 3) + (5 * 8)?")print(str(response))

Added user message to memory: What is (121 * 3) + (5 * 8)?
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
=== Calling Function ===
Calling function: multiply with args: {"a": 5, "b": 8}
=== Calling Function ===
Calling function: add with args: {"a": 363, "b": 40}
assistant: The result of (121 * 3) + (5 * 8) is 403.


## 在RAG管道上构建Mistral代理

在一个简单的10K文档上构建一个Mistral代理。我们使用Mistral嵌入和mistral-medium来构建RAG管道，并将其作为工具传递给Mistral代理。


In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'

--2024-03-23 11:13:41--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.09s   

2024-03-23 11:13:41 (19.3 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]



In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadatafrom llama_index.core import SimpleDirectoryReader, VectorStoreIndexfrom llama_index.embeddings.mistralai import MistralAIEmbeddingfrom llama_index.llms.mistralai import MistralAIembed_model = MistralAIEmbedding()query_llm = MistralAI(model="mistral-medium")# 加载数据uber_docs = SimpleDirectoryReader(    input_files=["./data/10k/uber_2021.pdf"]).load_data()# 构建索引uber_index = VectorStoreIndex.from_documents(    uber_docs, embed_model=embed_model)uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=query_llm)query_engine_tool = QueryEngineTool(    query_engine=uber_engine,    metadata=ToolMetadata(        name="uber_10k",        description=(            "提供2021年Uber财务信息。"            "使用详细的纯文本问题作为工具的输入。"        ),    ),)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    [query_engine_tool], llm=llm, verbose=True
)
agent = agent_worker.as_agent()

In [ ]:
response = agent.chat(
    "Tell me both the risk factors and tailwinds for Uber? Do two parallel tool calls."
)
print(str(response))

Added user message to memory: Tell me both the risk factors and tailwinds for Uber? Do two parallel tool calls.
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What are the risk factors for Uber in 2021?"}
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What are the tailwinds for Uber in 2021?"}
assistant: Based on the information provided, here are the risk factors for Uber in 2021:

1. Failure to offer or develop autonomous vehicle technologies, which could result in inferior performance or safety concerns compared to competitors.
2. Dependence on high-quality personnel and the potential impact of attrition or unsuccessful succession planning on the business.
3. Security or data privacy breaches, unauthorized access, or destruction of proprietary, employee, or user data.
4. Cyberattacks, such as malware, ransomware, viruses, spamming, and phishing attacks, which could harm the company's reputation and operations.
5. Climate change r